In [15]:
%matplotlib inline

import numba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
the_answer = "Follow your heart :-)"

## About the approach...

* The main assumption of this approach is: **similar users rate similar movies similarly.**
* It only takes into account the __rating__ a given user $u$ and the rating he gave to the movie $m$

## Loading 1M movielens dataset

In [7]:
path = 'ml-1m/'

In [8]:
movies = pd.read_csv(path+'movies.dat', sep='::', engine='python')
ratings = pd.read_csv(path+'ratings.dat', sep='::', engine='python')
users = pd.read_csv(path+'users.dat', sep='::', engine='python')

In [9]:
uid = ratings['UserID'].values - 1
mid = ratings['MovieID'].values - 1
rt =  ratings['Rating'].values

In [10]:
print("maximum user id: {0}".format(uid.max()))
print("maximum movie id: {0}".format(mid.max()))

maximum user id: 6039
maximum movie id: 3951


In [11]:
# Ratings matrix
ratings_matrix = np.zeros((uid.max()+1,mid.max()+1))
for i in range(len(rt)):
    ratings_matrix[uid[i],mid[i]] = rt[i]

qs = 2
query = ratings_matrix[-qs::]
ratings_matrix = ratings_matrix[0:-qs]

In [10]:
@numba.jit()
def minkowsky(ratings1, ratings2, k=3):
    """
    Computes the Minkowski distance.
    """
    mask1 = ratings1>0
    mask2 = ratings2>0
    mask = np.logical_and(mask1,mask2)
    n = len(ratings1)
    
    if not np.any(mask):
        return np.inf
    
    d = 0
    for i in range(n):
        if mask[i]:
            d += np.abs((ratings1[i]-ratings2[i]))**3
    return d**(1./3)
    

def mydist(x, y):
    return np.sum(x*y) / np.sqrt(np.sum(x**2))* np.sqrt(np.sum(y**2))

In [11]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree', metric=minkowsky)
nbrs.fit(ratings_matrix)

NearestNeighbors(algorithm='ball_tree', leaf_size=30,
         metric=CPUDispatcher(<function minkowsky at 0x125ec97b8>),
         metric_params=None, n_jobs=1, n_neighbors=10, p=2, radius=1.0)

In [110]:
def predict(ratings_matrix, users, movie, nbrs, k1=1000, k2=5):
    indexes = nbrs.kneighbors( users, k1, return_distance=False )[0]
    rating = 0.
    n = 0
    for index in indexes:
        if ratings_matrix[index,movie]==0:
            continue
        rating += ratings_matrix[index,movie] 
        n += 1
        if n==k2: break
    if n==0:
        return 0.
    return rating/float(n)

In [111]:
class KNNPredictor():
    def __init__(self):
        self.k = 10
    
    def predict(self, query_data):
        indexes = nbrs.kneighbors(query_data, self.k, return_distance=False)
        rating = 0.
        n = 0
        for index in indexes:
            rating += ratings_matrix[index,movie]
            if ratings_matrix[index,movie] > 0: 
                n += 1
        if n==0:
            return 0.
        return rating/float(n)   

## The problems...
* When we compute the $k$-nearest neighbors for a given user $u$, they might not have seen the movie $m$ we are trying to predict
* Then we need to find the $k$-nearest __neighbors that have also seen the movie__... 
* In some __pathological cases__ a user $u$ might not have $k$-neighbor that have seen the movie $m$...


In this approach __the user to query could (or could not) be__ in the training data. This is not possible in the other approaches (linear regression and MLP).

***
***
***

Whit time to fix it, it would look like this:

https://github.com/nchah/movielens-recommender

In [14]:
!python knn-recommend.py

===== 0 ==================================================
User # 3320 , distance: 1.09225018729
Highlander III: The Sorcerer (1994) 1    YOUR: 1
Boxing Helena (1993) 1    YOUR: 1
Pretty Woman (1990) 2    YOUR: 2
Close Shave, A (1995) 5    YOUR: 5
Michael Collins (1996) 4    YOUR: 4
Wrong Trousers, The (1993) 5    YOUR: 5
Amistad (1997) 4    YOUR: 3
===== 1 ==================================================
User # 2825 , distance: 1.24880819811
Amistad (1997) 3    YOUR: 3
English Patient, The (1996) 4    YOUR: 5
Wrong Trousers, The (1993) 5    YOUR: 5
Death and the Maiden (1994) 5    YOUR: 5
Lawrence of Arabia (1962) 4    YOUR: 4
Close Shave, A (1995) 5    YOUR: 5
Piano, The (1993) 5    YOUR: 4
===== 2 ==================================================
User # 1205 , distance: 1.41068360252
Sliding Doors (1998) 4    YOUR: 3
English Patient, The (1996) 4    YOUR: 5
Michael Collins (1996) 4    YOUR: 4
Close Shave, A (1995) 5    YOUR: 5
Wrong Trousers, The (1993) 5    YOUR: 5
Piano, The (1

***
***
***

## What is the best way to get a movie recommendation?

In [ ]:
print(the_answer)